# **Food-101 Image Classification**

## **Installation and import of libraries**

In [ ]:
!pip install mlflow

In [ ]:
!pip install mlflow dagshub

In [ ]:
## Import all libraries needed

import os

import ast
import shutil
import urllib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image

from typing import Tuple, Dict, Any, List
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from PIL import Image
from typing import Tuple, List

from matplotlib.pyplot import imshow
torch.set_grad_enabled(True)
%matplotlib inline

import mlflow
import dagshub

In [ ]:
dagshub.init("taed2-Food_Classification", "violeta51", mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/violeta51/taed2-Food_Classification.mlflow')
mlflow.set_experiment(experiment_name="REPORT INICIAL")

## **Initialization**

Let's set the working enviroment.

In [ ]:
# set a seed for garantee the replication once we finish get the model

seed = 7767
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

Since the model is fairly complex, training it in the GPU will save us time.

In [ ]:
# we select to work on GPU

if not torch.cuda.is_available():
       raise RuntimeError("You should enable GPU runtime!!")
device = torch.device("cuda")

In [ ]:
# Let's define some hyper-parameters

hparams = {
    'log_interval': 200,
    'epochs' : 20,
    'batch_size' : 64,
}

mlflow.log_param("log_interval", hparams["log_interval"])
mlflow.log_param("epochs", hparams["epochs"])
mlflow.log_param("batch_size", hparams["batch_size"])

## **Load the Data**
Since the training data are off-line, we need to load them.

In [ ]:
## Open an image

Image.open('/kaggle/input/data-food/external/apple_pie/1005649.jpg')

In [ ]:
## img dir: directory containing the images
## annotations file: file containing the labels

## the following code is a modified version of 
## the code extracted from https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self): # returns the number of samples in our dataset.
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        """if len(img_path)< 6:
            print(img_path)"""
            
        image = read_image(img_path)

        to_pil = transforms.ToPILImage()
        image = to_pil(image)
        
        width, height = image.size
        
        if width < 224 or height < 224:
            scale_factor = 512 / max(width, height)
    
        # calculate the new dimensions
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            image = image.resize((new_height, new_width))
        
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

In [ ]:
# define the composed transformation for training
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(),])
mlflow.log_param("data_augmentation", "RandomResizedCrop(224), RandomHorizontalFlip(p=0.5)")

# define the composed transformation for validation
# val_transforms = transforms.Compose([transforms.RandomResizedCrop(256),transforms.CenterCrop(224),transforms.ToTensor()])

In [ ]:
## read the folders and load them in a specific class structure

first = False
first_first = True

# transform the target 'words' into numerical values, this dictionary will associate these two values
dictionary = {} 
idx = 0

for dirname, _, filenames in os.walk('/kaggle/input/data-food/external/'):

    if first == True:
        label = dirname[len('/kaggle/input/data-food/external/'):]
        columns = ['file', 'label']
        target = pd.DataFrame(columns=columns)
        
        for filename in filenames:
            target.loc[len(target)] = [filename, idx]
        
        dictionary[idx] =  label
        sub_dataset = CustomImageDataset(target, dirname, transform = train_transforms)
        
        if first_first == True:
            train_data = sub_dataset
            first_first = False
        else:
            train_data = ConcatDataset([train_data, sub_dataset])
        
        idx = idx + 1
        
    first = True     

In [ ]:
# show a sample
train_data[2]

In [ ]:
# split into training and validation set
train_subset, val_subset = torch.utils.data.random_split(
train_data, [20000, 10000], generator=torch.Generator().manual_seed(7767))

In [ ]:
val_subset[14]

### Data Loader 
Once the data are loaded in the workspace, we can prepare the data to feed them into the model.

In [ ]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(
    train_subset, 
    batch_size=hparams["batch_size"], 
    shuffle=True,
    num_workers=1, 
    pin_memory=True,
)

val_loader = torch.utils.data.DataLoader(
    val_subset,
    batch_size=hparams["batch_size"],
    shuffle=False, 
    num_workers=1,
)

## **Training**

### Utils: useful functions for the training
Some useful function will needed later. Here we include some definition of them. 

In [ ]:
def adjust_learning_rate(
        optimizer: torch.optim, 
        epoch: int, 
        original_lr: float
        ) -> None:
    
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = original_lr * (0.1 ** (epoch // 30))
    # For some models, different parameters are in different groups with different lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def compute_accuracy(predicted_batch: torch.Tensor, label_batch: torch.Tensor) -> float:
    """
    Define the Accuracy metric in the function below by:
      (1) obtain the maximum for each predicted element in the batch to get the
        class (it is the maximum index of the num_classes array per batch sample)
        (look at torch.argmax in the PyTorch documentation)
      (2) compare the predicted class index with the index in its corresponding
        neighbor within label_batch
      (3) sum up the number of affirmative comparisons and return the summation

    Parameters:
    -----------
    predicted_batch: torch.Tensor shape: [BATCH_SIZE, N_CLASSES]
        Batch of predictions
    label_batch: torch.Tensor shape: [BATCH_SIZE, 1]
        Batch of labels / ground truths.
    """
    pred = predicted_batch.argmax(dim=1, keepdim=True) # get the index of the max log-probability 
    acum = pred.eq(label_batch.view_as(pred)).sum().item()
    return acum


def save_checkpoint(
        state: 'dict', 
        is_best: bool, 
        filename: str = 'checkpoint.pth.tar'
        ) -> None:
    
    torch.save(state, filename)
    
    # save an extra copy if it is the best model yet
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')  

### Define the training



In [ ]:
# obtain the model from pytorch
model = models.resnet50()

# declare Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
mlflow.log_param("optimizer", optimizer)

# define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_epoch(
        train_loader: torch.utils.data.DataLoader, 
        model: torch.nn.Module, 
        optimizer: torch.optim,
        criterion: torch.nn, 
        epoch: int,
        log_interval: int,
        device: torch.device
        ) -> Tuple[float, float]:

    # switch to train mode (activate the train=True flag inside the model)
    model.train()
    
    train_loss = []
    acc = 0.
    avg_weight = 0.1
    
    for i in range(len(train_loader)):
        batch = next(iter(train_loader))
        images = batch[0]
        target = batch[1]
        
        # set network gradients to 0.
        optimizer.zero_grad()

        # move images to gpu
        images = images.to(device)
        target = target.to(device)

        # forward batch of images through the network
        output = model(images)
        loss = criterion(output, target) # Compute the loss
        #mlflow.log_metric("loss_train",loss)

        # compute gradient and do SGD step
        loss.backward()
        optimizer.step()
        
        # compute metrics
        acc += compute_accuracy(output, target)
        train_loss.append(loss.item())
        
        # measure accuracy
        #acc1, acc5 = accuracy(output, target, topk=(1, 5))
        #mlflow.log_metric("accuracy_train_acc1",acc1)
        #mlflow.log_metric("accuracy_train_acc5",acc5)

        if i % log_interval == 0 or i >= len(train_loader)-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(images), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))
    avg_acc = 100. * acc / len(train_loader.dataset)
    
    return np.mean(train_loss), avg_acc 

In [ ]:
@torch.no_grad() # decorator: avoid computing gradients
def eval_epoch(
        test_loader: torch.utils.data.DataLoader,
        model: torch.nn.Module,
        criterion: torch.nn.functional,
        ) -> Tuple[float, float]:

    # Dectivate the train=True flag inside the model
    model.eval()
    
    test_loss = 0
    acc = 0
    for i in range(len(test_loader)):
        batch = next(iter(test_loader))
        data = batch[0]
        target = batch[1]
        
        data, target = data.to(device), target.to(device)

        output = model(data)

        # Apply the loss criterion and accumulate the loss
        test_loss += criterion(output, target).item()

        # compute number of correct predictions in the batch
        acc += compute_accuracy(output, target)

    test_loss /= len(test_loader)
    # Average accuracy across all correct predictions batches now
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc      # ENS RETORNA LES MÈTRIQUES EN LA VALIDACIÓ EN FUNCIÓ DE LES ÈPOQUES

In [ ]:
def train_net(
        network: torch.nn.Module,
        train_loader: torch.utils.data.DataLoader,
        eval_loader: torch.utils.data.DataLoader,    
        optimizer: torch.optim,
        num_epochs: int,
        plot: bool=True,
        ) -> Dict[str, List[float]]:
    
    """ Function that trains and evals a network for num_epochs,
      showing the plot of losses and accs and returning them.
    """
    tr_losses = []
    tr_accs = []
    te_losses = []
    te_accs = []

    network.to(device)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        tr_loss, tr_acc = train_epoch(train_loader, network, optimizer, criterion, epoch, hparams["log_interval"],device)
        mlflow.log_metric("loss_train", tr_loss, step=epoch)
        mlflow.log_metric("accuracy_train", tr_acc, step=epoch)
        te_loss, te_acc = eval_epoch(eval_loader, network, criterion)
        mlflow.log_metric("loss_eval", te_loss, step=epoch)
        mlflow.log_metric("accuracy_eval", te_acc, step=epoch)
        te_losses.append(te_loss)
        te_accs.append(te_acc)
        tr_losses.append(tr_loss)
        tr_accs.append(tr_acc)
    rets = {'tr_losses':tr_losses, 'te_losses':te_losses,
          'tr_accs':tr_accs, 'te_accs':te_accs}
    if plot:
        plt.figure(figsize=(10, 8))
        plt.subplot(2,1,1)
        plt.xlabel('Epoch')
        plt.ylabel('NLLLoss')
        plt.plot(tr_losses, label='train')
        plt.plot(te_losses, label='eval')
        plt.legend()
        plt.subplot(2,1,2)
        plt.xlabel('Epoch')
        plt.ylabel('Eval Accuracy [%]')
        plt.plot(tr_accs, label='train')
        plt.plot(te_accs, label='eval')
        plt.legend()
    return rets

### Model Parameters

In [ ]:
def get_nn_nparams(net: torch.nn.Module) -> int:
  """
  Function that returns all parameters regardless of the require_grad value.
  https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6
  """
  return sum([torch.numel(p) for p in list(net.parameters())])

In [ ]:
# Let's see the number of parameters containing in our network
print("Number of parameters:", get_nn_nparams(model))

### Train the model and check its performance

In [ ]:
model_log = train_net(model, train_loader, val_loader, optimizer, hparams["epochs"])

In [ ]:
model_log

In [ ]:
mlflow.end_run()